In [92]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from flask import jsonify,send_file
import pickle
import json
from tensorflow.keras.models import load_model
from joblib import dump,load
from sklearn.model_selection import KFold,cross_val_score
import joblib

In [93]:
df=pd.read_csv("D:\\genomic prediction\\ui\\data\\ddata.csv") 
df=df.drop(columns=['Unnamed: 0'])
x=df.iloc[:,:-2]
y=df.iloc[:,-2]
z=df.iloc[:,-1]
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(x, y)

model_r=RandomForestClassifier(n_estimators=7,max_depth=9,random_state=42,criterion="gini",max_features='sqrt',max_leaf_nodes=30) 

cv = KFold(n_splits=10, random_state=1, shuffle=True)
scores = cross_val_score(model_r, X_resampled, y_resampled, scoring='accuracy', cv=cv, n_jobs=-1)
print('Accuracy: %.3f (%.3f)' % (np.mean(scores), np.std(scores)))


"""
x_train,x_test,y_train,y_test=train_test_split(X_resampled,y_resampled,random_state=42,test_size=0.33) 
model_r.fit(x_train,y_train)
s_pred=model_r.predict(x) 
r_pred=model_r.predict(x_test)
print(accuracy_score(y_test,r_pred))

"""


Accuracy: 0.719 (0.039)


'\nx_train,x_test,y_train,y_test=train_test_split(X_resampled,y_resampled,random_state=42,test_size=0.33) \nmodel_r.fit(x_train,y_train)\ns_pred=model_r.predict(x) \nr_pred=model_r.predict(x_test)\nprint(accuracy_score(y_test,r_pred))\n\n'

In [94]:
model_r.fit( X_resampled,y_resampled)
joblib.dump(model_r, 'model_r.pkl')

['model_r.pkl']

In [95]:
s_pred=model_r.predict(x) 

In [96]:
s_pred=list(s_pred)
df_h=pd.DataFrame(df.iloc[:,:-2]) 
df_h['sub']=s_pred 
df_h['sub']=df_h['sub'].astype("category").cat.codes +1  
df_h['height']=z
df_h.columns=df_h.columns.astype(str)   

def Outlier_Treatment(df1,variable):
    percentile25=df1[variable].quantile(0.25)  #q1
    percentile75=df1[variable].quantile(0.75)  #q3
    iqr=percentile75-percentile25
    upper_limit=percentile75+1.5*iqr
    lower_limit=percentile25-1.5*iqr
    df_final = df1[ (df1[variable]>=lower_limit) & (df1[variable]<=upper_limit) ]
    return df_final

df_final=Outlier_Treatment(df_h,'height')

xh=df_final.iloc[:,:-1]
yh=df_final.iloc[:,-1] 




In [97]:
xh=xh.values
yh=yh.values
print(xh.shape,yh.shape)

(528, 10) (528,)


In [98]:
xh[0][9]

1.0

In [99]:
from sklearn.ensemble import RandomForestRegressor  
from sklearn.ensemble import BaggingRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
scaler=MinMaxScaler()
model_rr=RandomForestRegressor(n_estimators=13,max_depth=4,random_state=42)
bg_model=BaggingRegressor(estimator=model_rr,n_estimators=2,random_state=42)

mse_scores=[]
kf = KFold(n_splits=10, random_state=42, shuffle=True)
for train_index, test_index in kf.split(xh):
    X_train, X_test = xh[train_index], xh[test_index]
    y_train, y_test = yh[train_index], yh[test_index]
    
    # Fit the scaler on training data
    #scaler.fit(X_train)
    
    # Transform the training and test data
    #X_train_scaled = scaler.transform(X_train)
    #X_test_scaled = scaler.transform(X_test)
    
    # Train the model
    bg_model.fit(X_train, y_train)
    
    # Predict and evaluate
    y_pred = bg_model.predict(X_test)
    msee = mae(y_test, y_pred)
    mse_scores.append(msee)


print('mae: %.3f (%.3f)' % (np.mean(mse_scores), np.std(mse_scores)))

"""
X_train,X_test,Y_train,Y_test=train_test_split(xh,yh,random_state=42,test_size=0.33)
from sklearn.preprocessing import MinMaxScaler
mm=MinMaxScaler()
mm.fit(X_train)
X_train=mm.transform(X_train)
X_test=mm.transform(X_test)
bg_model.fit(X_train,Y_train)
"""
 

mae: 16.512 (1.350)


'\nX_train,X_test,Y_train,Y_test=train_test_split(xh,yh,random_state=42,test_size=0.33)\nfrom sklearn.preprocessing import MinMaxScaler\nmm=MinMaxScaler()\nmm.fit(X_train)\nX_train=mm.transform(X_train)\nX_test=mm.transform(X_test)\nbg_model.fit(X_train,Y_train)\n'

In [102]:
type(xh)

numpy.ndarray

In [100]:
#scaler.fit(xh)
#bigx= scaler.transform(xh)
bg_model.fit(xh,yh)
#joblib.dump(bg_model,'model_br.pkl')
#joblib.dump(scaler,'scaler_x.pkl')

['model_br.pkl']